<a href="https://colab.research.google.com/github/liuqing321/-Amazon_Vine_Analysis/blob/main/Amazon_Reviews_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-17 03:54:20--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-12-17 03:54:20 (11.5 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://egbucket01.s3.us-east-2.amazonaws.com/amazon_reviews_us_Video_Games_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)","customer_count")
customers_df.show(20)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
|    2574873|             1|
|    4696154|             1|
|    5621202|             1|
|    5871933|             2|
|   44089812|             1|
|    2845910|             1|
|    5274369|             1|
|   39069693|             2|
|     137793|             1|
|   31914942|             3|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates(["product_id"])
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0439394422|school house rock...|
|9861064222|Black Nintendo Wi...|
|B00000K1XN|        WWF Attitude|
|B00001OX40|   Neo Cherry Master|
|B00001QEQD|     USCF Chess - PC|
|B00001XDTO|        Yoda Stories|
|B00001ZWQ0|Legend of the Riv...|
|B0000296ZL|      Game Boy Color|
|B00002SFMS|     Boss Rally - PC|
|B00002SSYX|Ristar (Sega Game...|
+----------+--------------------+
only showing top 10 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(20)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-08-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-08-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-08-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890| 2015-08-31|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651| 2015-08-31|
|R1VTIA3JTYBY02|     106569|B00008KTNN|     384411423| 2015-08-31|
|R29DOU8791QZL8|   48269642|B000A3IA0Y|     472622859| 2015-08

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [10]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
vote_df = df.filter("total_votes>=20").show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [ ]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_df = df.filter(("helpful_votes/total_votes>=0.5 and total_votes>=20")).show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [11]:
#  retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'
nonvine_df=df.filter(("helpful_votes/total_votes>=0.5 and total_votes>=20")).where("vine=='N'").show(20)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [12]:
# retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_df=df.filter(("helpful_votes/total_votes>=0.5 and total_votes>=20")).where("vine=='Y'").show(20)


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48771843|R3KKUSGFZWSUIY|B00YXO5UXG|     555176890|Turtle Beach - Ea...|     Video Games|          5|           56|         63|   Y|                N|Quality Chat Headset|Not every situati...| 2015-08-04|
|         US|   53080186|R10FO5UKKVZBK2|B00XO041RQ|     238654494|PDP AG7 True Wire...|     Video Games|          3|    

In [13]:
import pandas as pd
helpful_review_df= df.filter(("helpful_votes/total_votes>=0.5 and total_votes>=20")).toPandas()


In [24]:
total_helpful_reviews = helpful_review_df["review_id"].nunique()
total_helpful_reviews

40565

In [14]:
fivestar_df = helpful_review_df[helpful_review_df['star_rating']==5]
fivestar_df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
7,US,52553659,R36O341WWXXKNP,B00ZSJU97C,838780144,Disney Infinity 3.0 Edition: Star Wars Saga Bu...,Video Games,5,28,31,N,N,Upgrade to Disney Infinity 3.0- Worth It,We have been playing Disney infinity since the...,2015-08-30
11,US,14073759,R29BOS5HMAY1LO,B00W435BL4,281495895,Madden NFL 16 - PlayStation 3,Video Games,5,88,110,N,N,The wife will never know....,"This game is great. I am a madden faithful, an...",2015-08-29
24,US,1674841,RDX2ZZ46AM343,B00O9JLAX4,63546783,PlayStation 4 Black Friday Bundle - Grand Thef...,Video Games,5,32,34,N,Y,I'm not a Amazon prime member but omg this is ...,WORTH THE MONEY!!!!! I'm not a Amazon prime me...,2015-08-28
25,US,11295708,R60GI4Z1CNGGV,B00P45U628,784469446,"GAEMS M155 15.5"" HD LED Performance Gaming Mon...",Video Games,5,17,20,N,Y,I love it,Had to sneak out of the house the other day wi...,2015-08-28
32,US,18176251,R2FAARI3JQO9XQ,B00VFT4T5M,936171118,New Nintendo 3ds Only Fire Emblem If Special E...,Video Games,5,29,30,N,Y,Special Edition Items and Overview,**You must have a 3ds that can play Japanese g...,2015-08-27


In [15]:
total_num_fivestar = fivestar_df["review_id"].nunique()

total_num_fivestar

15711

In [16]:

vine_pd_df=df.filter(("helpful_votes/total_votes>=0.5 and total_votes>=20")).where("vine=='Y'").toPandas()
vine_pd_df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,48771843,R3KKUSGFZWSUIY,B00YXO5UXG,555176890,Turtle Beach - Ear Force Recon 30X Chat Commun...,Video Games,5,56,63,Y,N,Quality Chat Headset,Not every situation calls for (expensive) surr...,2015-08-04
1,US,53080186,R10FO5UKKVZBK2,B00XO041RQ,238654494,PDP AG7 True Wireless Headset for Xbox One,Video Games,3,23,23,Y,N,"Great sounding headphones, but microphone volu...",[[VIDEOID:a9ebb8e7454df6453f81e67fa32070eb]] T...,2015-08-03
2,US,29202184,RM4KSGEOR7MU1,B00YXO5X4M,832423177,Turtle Beach - Ear Force Stealth 420X Fully Wi...,Video Games,5,19,24,Y,N,Definitely Worth The Money,[[VIDEOID:5a58574fde2e94d8d9512d9b7d589e5a]] I...,2015-07-03
3,US,42836457,RG7VRMYLEXD23,B00P45U43O,231578141,Antec X-1 Cooler for Xbox One,Video Games,4,22,26,Y,N,A whisper quiet cooling exhaust fan for the Xb...,The Antec X-1 Cooler is basically a large fan ...,2015-04-16
4,US,24021862,R11O4YSCPSNL6L,B00MNPDMEQ,925909080,Turtle Beach - Ear Force Elite 800 - Premium F...,Video Games,3,20,26,Y,N,"A high-end gaming headset from Turtle Beach, b...",As a DJ and avid gamer I'm always in the marke...,2015-02-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,US,52422537,RLPTVGLU0JQIP,B001EQPDNS,231258334,Amazing Adventures: The Forgotten Ruins - Nint...,Video Games,3,42,45,Y,N,Not an Adventure--more like I Spy--,Amazing Adventures: The Forgotten Ruins For ...,2008-11-30
90,US,33354780,R3ASJ9SENYYYI0,B0018BELN2,794843117,Buzz! Quiz TV - Playstation 3,Video Games,5,40,46,Y,N,Find out what all the Buzz! is about!,"Tired of \\""You Don't Know Jack\\""? Bored wi...",2008-10-26
91,US,52422920,RNU8PK609WT6P,B0015HYPOO,588916709,Logitech PlayStation 3 Driving Force GT Racing...,Video Games,4,347,362,Y,N,Experience Depends on the Game... Wheel is Gre...,Summary [8.5 out of 10]<br />=================...,2008-07-11
92,US,34487454,R8YT75NJW0CM9,B000ME25P2,708426912,My Word Coach,Video Games,4,37,40,Y,N,My Word Coach - Cool Vocabulary Game with Nice...,Ubisoft has created a new line of language gam...,2008-01-16


In [27]:
import numpy as np
# Total number of reviews were that written as part of the Vine program (paid)
total_vine_review=vine_pd_df["review_id"].nunique()
total_vine_review

94

In [18]:
# Total number of 5-star reviews that were written as part of the Vine program (paid)
vine_fivestar = vine_pd_df[vine_pd_df['star_rating']==5]
paid_fivestar = vine_fivestar["review_id"].nunique()
paid_fivestar

48

In [19]:

nonvine_pd_df=df.filter(("helpful_votes/total_votes>=0.5 and total_votes>=20")).where("vine=='N'").toPandas()
nonvine_pd_df


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,7142190,R4PKAZRQJJX14,B00QZLVCU0,210935604,Valve - DOTA 2 Steam Wallet Card ($20),Video Games,1,21,34,N,N,What store doesn't sell gift cards that the ex...,Who pays 4 dollars more for a $20 gift card?<b...,2015-08-31
1,US,1085641,R2CI0Y288CC7E2,B00RHI62GY,626589765,ONE PIECE Pirate Warriors 3,Video Games,1,21,35,N,Y,i request change article,"Yesterday i received my package, however, the ...",2015-08-31
2,US,34170274,R127WEQY2FM1T3,B00W435BU0,487294121,Madden NFL 16,Video Games,1,147,175,N,Y,Hot Mess. Don't Buy.,This is an absolute hot mess. EA has become i...,2015-08-31
3,US,37932477,R2FJ94555FZH32,B00WJ0Z194,116945357,Infinity 3.0 Starter Pack,Video Games,2,55,60,N,N,"Some improvements on 2.0, but many game-breaki...",While Infinity 3.0 improves upon 2.0 in a numb...,2015-08-31
4,US,45782027,R1U3AR67RE273L,B00W435C0Y,406569386,Madden NFL 16,Video Games,1,51,65,N,Y,CAN'T PLAY - NO EA TIMETABLE FOR PATCH,I've owned a version of this game every year s...,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40466,US,51007632,RCMDCDJR16IKW,B00000I1BQ,204465540,Crash Bandicoot,Video Games,4,27,30,N,N,Sony has an excellent response to Sonic the He...,Because it is a CD based unit a lot of the gam...,1999-10-13
40467,US,52014103,R3GYUWLD9FWCPS,B00000T94P,320623286,Dreamcast TremorPak,Video Games,5,30,30,N,N,The Best of the Rumble Accessories for the Dre...,Dreamcast.ign.com did a comparison of the Perf...,1999-10-01
40468,US,52576981,RU0J1ZMBCLD27,B00000K2XJ,83712832,Darkstone - PC,Video Games,5,27,29,N,N,The best Action/RPG this year!,Darkstone is the best Action/RPG of 1999. The...,1999-09-06
40469,US,51635190,R1D69XH2THYKTG,B00000K13J,607824256,InterAct S-Video Cable,Video Games,5,18,20,N,N,Must Have For Clear Picture!,This is a must have accessory if your TV has a...,1999-08-30


In [20]:
#Total number of reviews that were not part of the Vine program (unpaid)
unpaid_number=nonvine_pd_df["review_id"].nunique()
unpaid_number 

40471

In [21]:
# Total number of 5-star reviews that were not part of the Vine program (unpaid)
nonvine_fivestar = nonvine_pd_df[nonvine_pd_df['star_rating']==5]
unpaid_fivestar=nonvine_fivestar["review_id"].nunique()
unpaid_fivestar

15663

In [25]:
# % of 5-star reivews without vine program 

percentage_of_unpaid_fivestar = (unpaid_fivestar/total_helpful_reviews)*100

percentage_of_unpaid_fivestar

38.612104030568226

In [26]:
# % of 5-start reviews with vine program 
percentage_of_paid_fivestar = (paid_fivestar/total_helpful_reviews)*100

percentage_of_paid_fivestar

0.11832860840626155

In [29]:
# Five_star rate in vine reviews

vine_rate = (paid_fivestar/total_vine_review)*100
vine_rate

51.06382978723404

In [31]:
# Five_star rate in non_vine reviews 

non_vine_rate = (unpaid_fivestar/unpaid_number)*100
non_vine_rate

38.701786464381904